<img src = "./bgsedsc_0.jpg">

$\newcommand{\bb}{\boldsymbol{\beta}}$
$\DeclareMathOperator{\Gau}{\mathcal{N}}$
$\newcommand{\bphi}{\boldsymbol \phi}$
$\newcommand{\bx}{\boldsymbol{x}}$
$\newcommand{\by}{\boldsymbol{y}}$
$\newcommand{\whbb}{\widehat{\bb}}$
$\newcommand{\hf}{\hat{f}}$
$\newcommand{\tf}{\tilde{f}}$
$\newcommand{\ybar}{\overline{y}}$
$\newcommand{\E}{\mathbb{E}}$
$\newcommand{\Var}{Var}$
$\newcommand{\Cov}{Cov}$
$\newcommand{\Cor}{Cor}$

# Tree-based & additive models, Ensemble methods: forests, bagging and boosting

This lecture introduces some fundamental concepts for effective predictive modelling based on the concept of decision trees. The combination of the models and algorithms developed here, combined also with linear models and regularisation, is what most often used in practice due to the resultant high prediction accuracy.  

As you will see within this paradigm the features become part of the learning procedure - not merely by selection, as e.g. with lasso


## Summary
This module introduces both classification and regression models under the umbrella of **decision trees**. Those are non-linear models that work by iteratively splitting the space of one input feature (explanatory variable) in a way that the resulting two splits in training data have minimum variability in terms of the output value to predict. Combining the sequences of splits that the algorithm finds, one can represent those models as trees with branches and leaves. 

In [ ]:
## We load the relevant modules

%matplotlib inline
import matplotlib.pylab as plt

from helper_functions import *

import pandas as pd
import seaborn as sns
import numpy as np
import sklearn

In [ ]:
# A helper function to display the tree.
# NOTE: requires pydotplus and graphviz libraries. 
#       for MACOS/LINUX just install by typing "conda install pydotplus"  (and graphviz too) at the terminal
#       for Windows install by typing "conda install pydotplus" at the Anaconda Powershell Prompt 
#       for Windows and graphviz, if conda install doesn't work, then download the zip file from https://graphviz.gitlab.io/_pages/Download/Download_windows.html
#             unzip, include the 'bin' folder path in an environment variable named 'path' hen do 'pip install graphviz' and 'conda install graphviz' from anaconda prompt
#             finally, also include in the environment variable 'path' the path to the Anaconda graphviz package, i.e. SomePath\Anaconda3\Library\bin\graphviz
    
from IPython.display import Image 
import pydotplus
import graphviz
def plot_tree(clf, feature_names, target_names):
    dot_data = sklearn.tree.export_graphviz(clf, out_file=None, 
                             feature_names=feature_names,  
                             class_names= target_names,  
                             filled=True, rounded=True,  
                             special_characters=True) 
    return pydotplus.graph_from_dot_data(dot_data).create_png() 


## Linear models with tree-based features

To work our way towards tree-based models consider the following linear model with **tree-based features** 

We have input $\bx=(x_1,\ldots,x_p)^T$ a vector of $p$ input variables. Then, 

$$f(\bx;\bb) = \sum_{m=1}^p \beta_m 1[\bx \in R_m]$$

where the $R_m$ are disjoint subsets of the input space defined by a binary tree, say splitting each variable on the median. 

For example say $p=2$ and we split each variable to the median, then there would be four regions: 
+ region 1: $x_1 < median(x_1)$ and $x_2 < median(x_2)$
+ region 2: $x_1 > median(x_1)$ and $x_2 < median(x_2)$
+ region 2: $x_1 < median(x_1)$ and $x_2 > median(x_2)$
+ region 2: $x_1 > median(x_1)$ and $x_2 > median(x_2)$

Below, there is a more generic partitioned space of 2 variables

<img src = "tree_subregions.png">

In terms of function approximation, the assumption here is that the regression function $f$ is constant in each of these regions, hence the data are split in homogenous subgroups. 

It is trivial to fit this regression model to data, each $\beta_m$ is estimated from the training that fall in the subregion alone, and equals the sample mean of the data in that subregion. Hence, this is a simple instance of a linear model. 

However, there are (at least) two main problems with this approach:

1. Computational: for $p$ variables there are $2^p$ regions if we split each variable in its median, hence we quickly end up with massive number of features - and will overfit unless we take precautions 
2. Predictive: even if we have carefully preselected the input variables in terms of their relevance for prediction it might turn out that a few, or even most, are not useful for prediction, hence we should not be subdiving the population according to these variables. On the other hand, for those that are relevant for prediction, there is no good reason why the median, or any preselected quantile, provides a good split. We would like to learn from the data itself where to split each variable (if used at all in the model); and maybe in order to get a good predictive model we might have to split some variables several times. Which now feeds back into the "computational" problem since the  potential regions might be much larger than $2^p$.

### Tree based models: a first illustration

Tree-based models, and the associated learning algorithms for fitting them to data, address these two challenges and result in a non-linear (in the parameters as well as the inputs) model where the variables to be split and the split points are learned from data. 

Before we discuss tree-based models and algorithms any further let's immediatelly implement one such algorithm to the spam training data. The objective of this particular example is to classify emails as 'spam' or not-spam ('email'), based on features that are basically the occurrence of certain words or expressions.

What we are doing below is **bad practice**: never run a function you do not know what it does - especially when it has many options

In [ ]:
# Load the spam
dataset = pd.read_csv("spam_small_train.csv")
X = dataset.drop('class', axis = 1)
y = dataset["class"]

# to print stats
feature_names = X.columns
class_labels = ["email", "spam"]

In [ ]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier()
model.fit(X,y)

In [ ]:
# plot the tree
Image(plot_tree(model, feature_names, class_labels))

Note the complexity of this tree. Forks to the right imply 'false' in the splitting criteria, forks to the left are 'true'. Orange colours mean output 'no spam', whereas blue colours mean 'spam'. The latter are predominant in the right hand side of the tree. You can zoom into the figure by saving (right-clicking) and opening elsewhere.

Before we go any further, lets check the in and out of sample performance of the model.

In [ ]:
# read the test data, extract the info and create predictions

spam_test = pd.read_csv("spam_small_test.csv")
Xtest = spam_test.drop("class",axis=1)
ytest = spam_test["class"]
pred = model.predict_proba(X)
test_pred = model.predict_proba(Xtest)

In [ ]:
plt.plot(pred[:,1],"o")

In [ ]:
plt.plot(test_pred[:,1],"o")

We observe that the model predicts 'spam' or 'email' with apparently high confidence on its results (probabilities close to 0 or 1, few points in between).

#### AUC recap
Recall that, for binary classification models, to check performance we use the Area Under the Curve (**AUC**), considering the **ROC** curve (Receiver Operating Charateristic). Remember that ROC is a probability curve and AUC represents degree or measure of separability. It tells how much model is capable of distinguishing between classes if we use different thresholds of probability. In general, the higher the AUC (from 0 to 1), the better the model is at predicting a binary classification (in this case 'spam' and 'email'. 

The curve plots the relationship between True Positive Rate and False Positive Rate, having these definitions:

$TPR=Sensitivity=Recall=\frac{TP}{TP+FN}$

$FPR=1-Specificity=\frac{FP}{FP+TN}$

If AUC is 1 (the maximum), it means we can isolate and fully predict one class (e.g. 'spam') without any missclassficaction (e.g. 'emails' classified as 'spam'). Normally, the cost of having high sensitivity (get all 'spam' classified correctly) is having also a relatively high FPR (classify incorrectly as 'spam' some 'email').

In [ ]:
# Custom plot function
get_auc(y, pred, class_labels, column=1, plot=True) # Helper function

In [ ]:
# Custom plot function
get_auc(ytest, test_pred, class_labels, column=1, plot=True) # Helper function

There are clear signs of overfitting here, because AUC is 1 for training set, but clearly smaller for test set.

## Understanding the model and the algorithm

From the Hastie et al. book, here is an example of the binary tree representation of the tree-based model and the corresponding partition of input space into subregions

<img src = "binary_tree.png"> <img src = "tree_subregions.png">

In this method to predictive modelling the model and the algorithms are pretty much interweaved. The loss function can be used pretty much as in our previous supervised learning approaches. However, here it is not entirely straightforward (but can be done!) to write down a model for $f$  and then define an algorithm that tries to minimize the loss function 

From a very high level we can think of the "model" as 

$$f(\bx;\bb) = \sum_{m=1}^p \beta_m 1[\bx \in R_m]$$

in which we are interested to learn both $p$ and the regions $R_i$. The resultant optimization problem is **non-convex** and rather than
parameterising the regions, one typically uses an algorithm that then dictates the regions. 

A common approach (CART) is to use **forward search**, a heuristic optimization approach that is also used for variable selection in regression:
+ To grow the tree one level more, all possible variables are considered, for each the optimal cutoff point is found and then the variable that achieves largest decrease in loss function is chosen to determine the next level
+ This is heuristic but very fast to implement

This approach is by no means immune to overfitting - and we have seen this already.
+ One approach is to bound the depth or the total number of leaves
+ Another is to use a **backward search** after the forward, effectively a pruning step that tries to remove branches. The backward criterion is effectively a penalized likelihood criterion based again on a **regularization parameter**. 
+ As usual, a predictive measure is used to estimate the regularizing parameter (depth/backward parameter), which again relies on some estimate of MSE - e.g. CV. 

### Behind the scenes

+ Non-convex optimisation and heuristics
  + forward-backward search
  +  model fit criteria for tree-based models (e.g. regression and classification)
  +  model complexity criteria for tree-based models; regularisation parameter
+ Variable selection and importance
+ Categorical predictors and missing data

Lets revisit the sklearn class DecisionTree

## Complexity, accuracy and overfitting

Here we show *overfitting* risk: allowing an increasing number of final leaves in our tree, we increase the accuracy on training data, but not necessarily increase accuracy when applying to new (test) data. 



In [ ]:
# some sklearn tools
from sklearn.metrics import accuracy_score

# keep the results in a list
complexity_value = []
test_accuracy = []
train_accuracy = []

# loop through possible values of max_leaf_nodes
for max_leaf_nodes in range (2, 200): 
    model  = DecisionTreeClassifier(criterion = "entropy", max_leaf_nodes=max_leaf_nodes) 
    model.fit(X, y)
    
    #predict both on train and test set
    y_pred = model.predict(Xtest)
    y_pred_train = model.predict(X)
    
    # store the data to be used for plotting
    train_accuracy.append(accuracy_score(y, y_pred_train))
    test_accuracy.append(accuracy_score(ytest, y_pred))
    complexity_value.append(max_leaf_nodes)
    

plt.plot(complexity_value, train_accuracy, label='Train accuracy')
plt.plot(complexity_value, test_accuracy, label='Test accuracy')
plt.xlabel("complexity")
plt.ylabel("Accuracy")
plt.legend()

## Implicit variable selection and scoring

The model and algorithm effectively do a variable selection, since some variables drop out (never chosen for split) and a scoring of those that enter the model. Before we go into the mechanics of these procedures, lets see some results for spam

In [ ]:
model  = DecisionTreeClassifier(max_leaf_nodes=20) 
model.fit(X, y)
important_features = pd.DataFrame(model.feature_importances_/model.feature_importances_.max() ,index=X.columns, columns=['importance'])
# it is common to normalize by the importance of the highest
important_features.sort_values('importance', ascending=False)


In [ ]:
# Lets see the tree too

Image(plot_tree(model, feature_names, class_labels))
# do right click save image as and open outside the notebook to see details

## Stability 

Lets see how stable the previous results are to different subsets of the data

In [ ]:
from sklearn.model_selection import train_test_split

test_accuracy_argmax = [] # the maximal test accuracy achieved for each split
importance_char = [] # the variable char_! importance

for bootsam in np.arange(100):
    # split randomly dataset; do not fix the seed to see variation
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    #
    # First search for depth 
    test_accuracy = []
    complexity_value = []
    for max_leaf_nodes in np.arange (5, 40): 
        model  = DecisionTreeClassifier(max_leaf_nodes=max_leaf_nodes) 
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        test_accuracy.append(accuracy_score(y_test, y_pred))
        complexity_value.append(max_leaf_nodes)
    test_accuracy_argmax.append(complexity_value[np.argmax(test_accuracy)])
    #
    #print(f"Optimum max leaf {complexity_value[np.argmax(test_accuracy)]}")
    # Then find and store the relative importance of fare for the chosen tree
    
    model  = DecisionTreeClassifier(max_leaf_nodes=complexity_value[np.argmax(test_accuracy)]) 
    model.fit(X_train, y_train)
    important_features = pd.DataFrame(model.feature_importances_/model.feature_importances_.max() ,index=X.columns, columns=['importance'])
    importance_char.append(important_features.loc["char_freq_!",:].values[0])
    
# Print the results in a convenient manner
result =pd.DataFrame(test_accuracy_argmax,columns=["depth"])
result["score_char"] = importance_char
result.plot(x="depth",y="score_char",kind="scatter")

result.describe()

## From tree to forest: Bagging and random forests

The earlier experiment that "growed" a tree-based model on different subsets of the data revealed an important aspect of tree-based models, that of *instability*, small changes in the data (e.g. removing 20% of observations) causes large changes in the learned model.

Here is another example with some simulated data from Hastie, Tibshirani and Friedman book


<img src="bagged_trees.png"> 

This is typical of an estimator with high variance

+ The bias-variance tradeoff in ML 

  + The population-averaged tree-based model 
  
B(ootstrap) AGG(regation) ING tries to estimate the population-averaged estimator by boostrapping the procedure and averaging across datasets. The resultant learned function is not anymore a decision tree, but a linear combination of trees. This makes bagging an ensemble method - effectively a *model averaging* approach. 


Bagging does not change the bias of the underlying estimator, but it can decrease its variance, hence improve prediction performance. 

This is what happens on the same simulated dataset of Hastie, Tibshirani and Friedman

<img src="bagged_trees_error.png">

The tree-based estimators from each bootstrap sample are correlated with each other. If they are significantly positively correlated we should not expect any advantages from bagging. Hence, we should try to produce tree-based estimators with as little correlation between them as possible. This is what *random forests* try and do, by forcing the growth of trees to evolve to different "species" across the different bootstrap samples. 

Estimators that exhibit significant instability can benefit from bagging and random forests since it is more likely that the results on different bootstrap samples will be considerably different from eachother. 

Algorithmically, *random forest* require a small change to the common forward-backward heuristic optimisation algorithm used for growining trees. At forward steps instead of considering all variables as candidates for splitting, choose a subset each time and consider only those.The resultant trees, one for each bootstrap sample, are averaged to produce the random forest estimator 

### Random forests in sklearn

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

np.random.seed(31415) # impose random seed for reproducibility

dataset = pd.read_csv("spam_small_train.csv")
X = dataset.drop(['class'], axis = 1)
y = dataset["class"]

# to print stats
feature_names = X.columns
class_labels = ["email", "spam"]


forest = RandomForestClassifier(n_estimators=20)
scores = cross_val_score(forest, X, y, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))


importances = forest.fit(X,y).feature_importances_
important_features = pd.Series(data=importances/importances.max() ,index=feature_names)
important_features.sort_values(ascending=False)




### Excercise 1
As an exercise, check how accuracy is affected when dropping the features that are less important (i.e. importance less than 0.05). Increase the reported decimals if needed.

In [ ]:
# Your code here



## Boosting 

This is another way to reduce the variance of an estimator and increase predictive accuracy. This has links to a number of key ideas in machine learning:

+ class imbalance in classification by "boosting", rather than the class, the data that is hard to classify
+ bagging, by doing various passes through the data and averaging the estimators
+ forward search for heuristic optimisation, as it turns out that a popular implementation does exactly that
+ linear models with regularisation, as it turns out it is a linear combination of features, partially linear in the parameters, with a regulariser for complexity 

It can be applied to many ML methods, but it works really well with trees and the combination of the two yields often the best performing approach to predictive modelling

### Boosting and additive models

The essense of the method is as follows. Say we have models $f_m(x,\beta_m)$, for example each could be a tree-based model. We wish to fit a linear combination of those to data, i.e., 

$$
f(x,\theta,\beta) = \sum_{m=1}^p \theta_m f_m(x,\beta_m)
$$

where $f_m$ stands for the *m_th* model and $\theta_m$ is the corresponding weight.

Following the fundamental principles of machine learning modelling we also need a data-function mismatch component in the model, for example: 

+ in regression $L(y_i,f(x_i)) = (y_i-f(x_i))^2$ is the deviance
+ in classification, with output coded as $y_i \in \{+1,-1\}$, $L(y_i,f(x_i)) = \log(1+e^{-y_i f(x_i,\beta)})$ is also the negative log-likelihood.

Therefore, to learn such additive model from data we need to solve 

$$
\arg \min_{\theta,\beta} \sum_{i=1}^n L\left( y_i, \sum_{m=1}^p \theta_m f_m(x,\beta_m) \right)
$$

In other words, we need to find threshold cuts, and weight of each classifier, so that overall we minimize the loss function.

This is a hard, non-convex optimization. We can see boosting and its various versions as iterative optimization algorithms for fitting such additive model to data. 

### Possibilities

+ Forward search for heuristic optimization: fit the model one term at a time. Combined with the exponential loss, $L(y_i,f(x_i)) = e^{-y_i f(x_i)}$, this yields the so-called *AdaBoost* (Adaptive Boosting). Main highlights of AdaBoost are that:

    + Each tree is a simple 'decision stump', so only one node per tree (really weak classifier);
    + Each iteration we re-weight observations, up-weighting the ones with lower accuracy in previous trees.

+ Gradient-based minimization: this yields *gradient boosting*.  Gradient boosting optimizes weights of observations in next tree by using gradients in the loss function, applied to all trees that have been trained up to that point. GBM divides the optimization problem into two parts by first determining the direction of the step and then optimizing the step length. 

+ Extreme Gradient Boosting (*XGBoost*): Similar to GBM but each iteration solves the gradient search in one single step using Taylor expansion of the gradient formula. On the other hand, it uses a more regularized model formalization to control over-fitting (l1, l2 parameters), which typically implies better performance.

The risk of overfitting on those models has to be treated by using some regularization parameters (e.g. to limit the number of leaves) 

In particular, there is an issue called over-specialization, which means that trees added at later iterations tend to impact the prediction of only a few observations, so that their contribution towards the rest of the dataset becomes negligible. This is only addressed by Extreme Gradient Boosting model, by using regularization.


Lets see boosting in action for the spam data

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

np.random.seed(3123) # impose random seed for reproducibility

# Uncomment to test the effect of removing less important variables
# X = dataset.drop(['class','word_freq_hpl','word_freq_re','word_freq_edu','word_freq_you','word_freq_our'], axis = 1)
# feature_names = X.columns

tree = GradientBoostingClassifier(n_estimators=50)
scores = cross_val_score(tree, X, y, cv=10,  scoring='roc_auc')
print("Accuracy: %0.3f (+/- %0.3f)" % (scores.mean(), scores.std() * 2))

importance = tree.fit(X,y).feature_importances_
important_features = pd.Series(data=importance/importance.max() ,index=feature_names)
important_features.sort_values(ascending=False)

### Exercise 2
As an exercise, check how accuracy is affected when dropping the features that are less important (i.e. importance less than 0.05). Increase the reported decimals if needed.

In [ ]:
# Your code here


## Some hints for practitioners

### Models
#### CART (single tree)
Use *sklearn* function $DecisionTreeClassifier$ for classification, or $DecisionTreeRegressor$ for regression. These models are simpler to understand and explore but typically yield worse results than the ones based on 'multiple tree' approach. Some interesting parameters  to tune (for classification) are:
+ *max_depth*: Number of levels in a tree
+ *min_samples_split*: Minimum number of samples left to try a new split
+ *min_samples_leaf*: Minimum number of samples allowed in a leaf
+ *min_impurity_decrease*: Don't accept the split if we don't reach a minimum decrease in impurity
+ *min_impurity_split*: Early stop criteria, don't split if mininum impurity has been reached
+ *class_weight*: Use 'balanced' if classes are unbalanced.


#### Random forest
Use *sklearn* function $RandomForestClassifier$ for classification, or $RandomForestRegressor$ for regression. Main parameters are:
+ *n_estimators*: Number of trees to build
+ *max_features*: Number of features to consider when looking for the best split. If less than 100%, this intruduces stochasticity.
+ *boostrap*: Keep default 'True' to allow boostrapping the sample for each tree, useful to prevent overfitting
+ *oob_score*: If True, yields a proxy for expected accuracy, using *out of bag* observations per tree.
+ *n_jobs*: Number of CPU cores to be used (parallelization).
+ *max_depth*, *min_samples_split*, *min_impurity_decrease*, *min_impurity_split*, *class_weight*: Similar to CART


#### AdaBoost
Use *sklearn* function $AdaBoostClassifier$ for classification, or $AdaBoostRegressor$ for regression. Main parameters are:
+ *n_estimators*: Number of trees to build
+ *learning_rate*: Intensity of the re-weighting (boosting) each time we build a new tree.
+ 'base_estimator': Weak learner, by default 'DecisionTreeClassifier(max_depth=1)', but one can for instance increase depth to check if this helps.

#### GradientBoosting (GBM)
Use *sklearn* function $GradientBoostingClassifier$ for classification, or $GradientBoostingRegressor$ for regression. Main parameters are:
+ *n_estimators*: Number of trees to build
+ *subsample*: Percentage of samples to use for every tree.
+ *learning_rate*: similar to AdaBoost
+ *max_features*,*max_depth*, *min_samples_split*, *min_impurity_decrease*, *min_impurity_split*, *class_weight*: Similar to random forest

#### Extreme Gradient Boosting (XGBoost)
Use *xgboost* package and follow documentation at https://xgboost.readthedocs.io/en/latest/python/

Main functions are $xgboost.XGBClassifier$ or $xgboost.XGBRegressor$, which are an API to be used with *sklearn*.

Main parameters are:
+ *gamma*: similar to min_impurity_decrease. Minimum loss reduction required to make a further partition on a leaf node of the tree.
+ *colsample_bytree*:  Percentage of features to consider when constructing each tree.
+ *reg_alpha*:  L1 regularization term on weights to control overfitting
+ *reg_lambda*: L2 regularization term on weights to control overtiffing
+ *n_estimators*,*subsample*, *learning_rate*,*max_depth*: Similar to Gradient Boosting. 

#### Other MART (Multiple Additive Regression Trees) models:
+ *ligthgbm* package: Computationally faster than GBM, https://lightgbm.readthedocs.io/en/latest/Python-Intro.html
+ *catboost* package: Computationally faster than GBM, allows using GPU, https://github.com/catboost/catboost and https://catboost.ai/
+ Extremely Randomized Trees: Similar to random forest but optimization of splits is based on discrete random thresholds instead of exploring the entire space, so it's faster (and sometimes more accurate since is less prone to overfit). Use *sklearn* function $ExtraTreesClassifier$ or *ExtraTreesRegressor*

### Hyperparameter optimization
Use *sklearn* functions such as $GridSearchCV$ or $RandomizedSearchCV$. See example above in random forest section.

## References

Hastie, T., Tibshirani, R., Friedman, J., 2009. *Elements of Statistical Learning*. 2nd Edition. Chapters 4.5, 9.1, 9.2, 10 and 15.  https://web.stanford.edu/~hastie/ElemStatLearn/

Chen, T., Guestrin, C., 2016. *XGBoost: A Scalable Tree Boosting System*. Proceedings at Conference of Knowledge Discovery and Data Mining, August 13-17, 2016, San Francisco, CA, USA. https://www.kdd.org/kdd2016/papers/files/rfp0697-chenAemb.pdf

XGBoost tutorials. https://xgboost.readthedocs.io/en/latest/tutorials/index.html